<a href="https://colab.research.google.com/github/stayxyz/Banking-Analytics-Basel-III-/blob/master/Banking_Analytics_(Basel_III).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#                         Appendix



## Colab Link https://colab.research.google.com/drive/1kJVibArE7uHwQLN2ykwNEXF7Pw9MQgjN

# 1.0 Initiate the Data File 

In [ ]:
import numpy as np
!gdown https://drive.google.com/uc?id=1dkEHvr6w6-4o4aTebCLwT6k0kI59tY0l
import pandas as pd
Bankdata = pd.ExcelFile('/content/Dataset.xlsx')
Bond = pd.read_excel(Bankdata, 'Bond')
Mortgage = pd.read_excel(Bankdata, 'Mortgage')
Loan = pd.read_excel(Bankdata, 'RetailLoan')

Downloading...
From: https://drive.google.com/uc?id=1dkEHvr6w6-4o4aTebCLwT6k0kI59tY0l
To: /content/Dataset.xlsx
17.0MB [00:00, 46.6MB/s]


#1.1 Calculate the Current Bond Price

##1.1.1 Functions to Calculate the Bond Price


$$
Bond Price = Coupon \cdot \frac{1-(1+r)^{-n}}{r} + \frac{FaceV}{(1+r)^{n}} 
$$
*where*
$$
Coupon = YearlyRate \cdot FaceValue
$$


$$
r = Discount Rate
$$

$$ n = Time2Maturity $$


$$FaceV = Principle$$


**Notice that for zero coupon bonds, "Remainning Coupon" = 0**
\

$$Zero Coupon Bond = \frac{FaceV}{(1+r)^{n}} $$







In [ ]:
# Function to determine discout rate 

def discount_deter (Maturity):
  #As of September 26th, 2019
  short_term_rate = 1.58/100  #1 to 3 years 
  medium_term_rate = 1.43/100 #3 to 5 years 
  relative_long_term_rate = 1.37/100 #5 to 10 years
  long_term_rate = 1.52/100 #over 10 years
  if Maturity <= 3: 
    disr =  short_term_rate
  elif (Maturity > 3) and (Maturity <= 5):
    disr =  medium_term_rate
  elif (Maturity >5) and (Maturity <= 10):
    disr =   relative_long_term_rate
  else:
    disr =  long_term_rate
    
  return disr

# Function to calculate the bond prices

def Price_cal (npay,yrate,Time,FaceV):
  
  # Handle 0 coupon bond
  if npay == 0: 
    disr = discount_deter(Time)
    output = FaceV/((1+disr) ** Time) 
    
  # Handle normal coupon bond
  elif npay > 0:
    disr = discount_deter(Time)
    coupon = yrate/100* FaceV
    output = -np.pv(disr,Time,coupon,FaceV)
  else : 
    print ('error')
  
  return output
    

## 1.1.2 Loop though the Data Set to Calculate the Bond Price 

In [ ]:
Stop = max(Bond.id)
CurrPrice = []
TotalPrice = []
for index, row in Bond.iterrows():
  if row['id'] <= Stop:
    price = Price_cal(row['CouponsRemaining'], row['YearlyRate'], 
                      row['Time2Maturity'], row['Principal'])
    CurrPrice.append(price)
    # Since Professor announced that those prices are in cents
    TotalPrice.append(price*row['BondsHeld']/100)
  else: 
    break
    
    
## Attach Saved result to data frame
Bond['CurrentPrice'] = CurrPrice
Bond['TotalPrice'] = TotalPrice
   

##1.1.3 Extract the Calculation Result 

In [ ]:
print(sum(Bond['TotalPrice']))

52605441714.425255


#1.2 Calculate the Provison & Capital Requirements in each product 

In this section, we will loop through the dataset of Bond, Mortgages, and Other Retail Loans. during the iterations, we will calculate the Provision, RWA under Standarized Appraoch, RWA under IRB Apporach, Capital Requirement under Standarized Appraoch, Capital Requirement under IRB Apporach for each business line. Some of the calculation results will be used in Section 1.3

##1.2.1 Initiate Constant Parameters and Functions that Required in the Calculation Process


Formulas below are the function we need to use to calculate the results


$$
Provision = PD \cdot LGD \cdot EAD$$


\

- EAD of Bond = Current Bond Price

- EAD of Mortgages = Unpaid Principle - Collectral 

- EAD of Loans = Unpaid Principle 

Where

$$ 
Unpaid Principle = Original Balance \cdot (1 + Period Rate) ^ {npaid} - Pmt \cdot [\frac{{(1 + Period Rate})^{npaid} - 1}{Period Rate}] $$

\

- PD = Max(Given PD, PD floor)

- LGD = Max(Given LGD, LGD floor)


\

Formulas for Calculating RWA and Capital Requirements:

$$
K = LGD \cdot \left\{ N\left( \sqrt{\frac{1}{1-R}} \cdot N^{-1}(PD) + \sqrt{\frac{R}{1-R}} \cdot N^{-1}(0.999) \right) - PD \right\} \left( \frac{1 + (M - 2.5)b}{1 - 1.5b}\right)
$$

Parameters for Mortgages
- Mortgages: $R = 0.15$

Parameters for Other Retail Loans

- Other retail: $R = 0.03 \left( \frac{1 - e^{-35PD}}{1 - e^{-35}} \right) + 0.16 \left( 1 - \frac{1 - e^{-35PD}}{1 - e^{-35}} \right)$

Parameters for Bonds
- Bond exposures $ R = 0.12 \left( \frac{1 - e^{-50PD}}{1 - e^{-50}} \right) + 0.24 \left( 1 - \frac{1 - e^{-50PD}}{1 - e^{-50}} \right)$

- $b  = (0.11852 - 0.05478 ln(PD))^{2} $

\

$$ RWA \space of \space IRB = 12.5 \cdot K \cdot EAD $$


$$ RWA \space of \space IRB  = RiskWeight \cdot EAD $$

**Risk Weight is assigned according to Chapter 3 OFSI**

\

$$ Capital \space Requirement \space  = RWA * Capital Requirement Percentage $$

\

For our bank $$Capital Requirement Percentage = 11.5 \% $$




In [ ]:
## According to the Foundational Approach, 
## LGD for Bonds is 75% if we consider them all as subordinated bonds
## So we assign the LGD for coperate bonds in the loop as well
Bond['LGD'] = 0.75

## For Retail Loans, the LGD are considered as 100% 
Loan['LGD'] = 1



## Assign PD and LGD Floor to each business line
## For Bonds and Retail Loans, since their default LGD is 75% abd 100% 
## respectively; therefore, LGD floor only applies to Mortgages
## We will apply the LGD and PD floors in the process of calculation instead of 
## alterning the data 
PDfloor = 0.03/100
LGDfloor = 0.10



# Capital Requirement Percentage 
cap_req_percent = 11.5 /100


#Formulas for calculate correaltion in different class of assets 
#Retail Exposures:
import numpy as np
def other_Retail_R (PD):
  output = 0.03 * ( (1 - np.exp(-35 * PD)) / (1 - np.exp(-35)) )
  output += 0.16 * (1 - ( (1 - np.exp(-35 * PD)) / (1 - np.exp(-35)) ) )
  return output


#Corporate Exposures

def Coper_b (PD):
  output = (0.11852 - 0.05478 * np.log(PD))**2
  return output

def Coper_R (PD):
  output = 0.12 * ( (1 - np.exp(-50 * PD)) / (1 - np.exp(-50)) )
  output += 0.24 * (1 - ( (1 - np.exp(-50 * PD)) / (1 - np.exp(-50)) ) )
  return output

#Function to calculate capital requirements for Bonds:

def CapReq_Bond(LGD, PD, M):
  from scipy.stats import norm
  b = Coper_b (PD)
  R = Coper_R (PD)
  K = norm.cdf(np.sqrt( (1 - R) ** (-1) ) * norm.ppf(PD) + 
        np.sqrt( R / (1 - R) ) * norm.ppf(0.999) ) - PD
  K *= LGD * ((1 + (M-2.5)*b)/(1-1.5*b))
  return(K) 
  
#Function to calculate capital requirements for Mortgages:
  
def CapReq_Morg(LGD, PD):
  from scipy.stats import norm
  R = 0.15
  K = norm.cdf(np.sqrt( (1 - R) ** (-1) ) * norm.ppf(PD) + 
               np.sqrt( R / (1 - R) ) * norm.ppf(0.999) ) - PD
  K *= LGD
  return (K)

#Function to calculate capital requirements for Other Retail Loan:

def CapReq_other_Retail(LGD, PD):
  from scipy.stats import norm
  R = other_Retail_R (PD)
  K = norm.cdf(np.sqrt( (1 - R) ** (-1) ) * norm.ppf(PD) + 
               np.sqrt( R / (1 - R) ) * norm.ppf(0.999) ) - PD
  K *= LGD
  return (K)






########################################################
########################################################
##
##
##
##    Standarized Risk Weight Constant
##
##
#######################################################
#######################################################


#Function to Assign weight to according to bond's rating 
# Weight For AAA to AA- : 20%
# Weight For A+ to A- : 50%
# Weight For BBB+ to BB- : 100%
# Weight For below B - : 150%
def Weight_assign(PD):
  if PD  <= 0.01:
    weight = 20/100
  elif (PD > 0.01) and (PD <= 0.12):
    weight = 50/100
  elif (PD > 0.12) and (PD <= 0.52):
    weight = 100/100
  else:
    weight = 150/100
    
  return weight


# Function to assign mortgages weights 
# (assume all mortgages are residential mortgages)
# If LTV ratio < 80% : risk weight is 35%
# If LTV ratio > 80%: risk weight is 75%

def Weight_assign_Mortgage (loan, collateral):
  if loan/collateral < 0.80:
    weight = 0.35 
  else:
    weight = 0.75
    
  return weight 


# Weight for Other Retail Loans are a constant
# defined as below 75%
Weight_Retail_Loan = 0.75





## 1.2.2 Calculate Provison and Capital Requirements in the Bond Business Line

In [ ]:
Stop1 = max(Bond.id)
#Initiate Empty List to record values 
Bond_Provison = []
Bond_IRB_RWA = []
Bond_IRB_Cap = []
Bond_weight = []
Bond_SA_RWA = []
Bond_SA_Cap = []

for index, row in Bond.iterrows():
  if row['id'] <= Stop1:
    # LGD floor and PD floor applies here
    adjusted_PD = max(row['PD'],PDfloor)
    adjusted_LGD = max(LGDfloor,row['LGD'])
    Provison = row['TotalPrice']*adjusted_PD*adjusted_LGD
    Bond_Provison.append(Provison)
     
    #RWA in F-IRB
    Cap_Req = CapReq_Bond(adjusted_LGD, adjusted_PD, row['Time2Maturity'])
    Cap_Req = Cap_Req * row['TotalPrice']
    RWA_V = 12.5 * Cap_Req
    Bond_IRB_RWA.append(RWA_V)
    
    #Capital Requirement in F-IRB
    Final_Cap_Req = RWA_V * cap_req_percent
    Bond_IRB_Cap.append(Final_Cap_Req)
    
    #Assign weight 
    weight_V = Weight_assign(row['PD'])
    Bond_weight.append(weight_V)
    
    #RWA in Standized Approach
    W_RWA_V = row['TotalPrice']  * weight_V
    Bond_SA_RWA.append(W_RWA_V)
    
    #Capital Requirement in SA
    Final_Cap_Req2 =  W_RWA_V * cap_req_percent
    Bond_SA_Cap.append (Final_Cap_Req2)
    
    
  else:
    break

    
## Attach Saved result to data frame
Bond['Bond_Provison'] = Bond_Provison
Bond['Capital_Requirement_IRB'] = Bond_IRB_Cap
Bond['IRB_RWA'] = Bond_IRB_RWA
Bond['weight'] = Bond_weight
Bond['SA_RWA'] = Bond_SA_RWA
Bond['Capital_Requirement_SA'] = Bond_SA_Cap



## 1.2.3 Calculate Provison and capital Requirements in the Mortgage Business Line

In [ ]:
## Provision in the Mortgage Business Line


## Function to Calculate the Remainning Balance
def Rem_Bal (Principal, APR, Years, npaid):
    monthly_interest = APR/ 12
    payment_number = Years * 12
    
    #Calculate the Fixed Payment of Mortgage
    monthly_payment = Principal*(monthly_interest/(1-(1+monthly_interest) 
                                                   **(- payment_number)))
    
    
    #Calculate Unpaid Priciple
    rem_bal = Principal * (1 - ((1 + monthly_interest) ** npaid - 1) / 
                           ((1 + monthly_interest) ** payment_number - 1))
    
    return rem_bal
    

## Calculation of Remaining Balance for Mortgages 
Stop2 = max(Mortgage.ID)
Mortgage_Provison = []
EAD = []
Remaining_Bal = []
Mortgage_IRB_RWA = []
Mortgage_IRB_Cap= []
Mortgage_weight = []
Mortgage_SA_RWA = []
Mortgage_SA_Cap = []

for index, row in Mortgage.iterrows():
  if row['ID'] <= Stop2:
    # Calculate Remaining Balance 
    Remaning_Bal_V = Rem_Bal(row['OriginalAmount'], row['Rate'],row['TermYears']
                             , row ['CurrentInstallment'])
    Remaining_Bal.append(Remaning_Bal_V)
    # Calculate the EAD 
    # If the collectral exceeds remaning balance, the EAD should be 0 
    # instead of negative
    EAD_V =  max(0, Remaning_Bal_V - row['Collateral_valuation'])
    EAD.append(EAD_V)
    # Calculate the Provison 
    # Applies PD and LGD Floor 
        
    adjusted_PD = max(row['PD'],PDfloor)
    adjusted_LGD = max(LGDfloor,row['LGD'])
    
    
    Provison = EAD_V * adjusted_PD * adjusted_LGD
    Mortgage_Provison.append(Provison)
    
    
    # Risk Weighted Assets in IRB 
    Cap_Req = CapReq_Morg(adjusted_LGD,adjusted_PD)
    Cap_Req = Cap_Req * EAD_V
    RWA_V = 12.5 * Cap_Req
    Mortgage_IRB_RWA.append(RWA_V)
    
    # Capital Requirement in IRB 
    Final_Cap_Req = RWA_V * cap_req_percent
    Mortgage_IRB_Cap.append(Final_Cap_Req)

    # Assign Weights to Mortgages
    Loan_V = row['OriginalAmount']
    Collateral_V = row['Collateral_valuation']
    weight_V = Weight_assign_Mortgage (Loan_V, Collateral_V)
    Mortgage_weight.append(weight_V)
    
    
    # Calculate RWA using SA
    W_RWA_V =  EAD_V * weight_V
    Mortgage_SA_RWA.append(W_RWA_V)
    
    # Capital Requirement in SA
    Final_Cap_Req2 = W_RWA_V * cap_req_percent
    Mortgage_SA_Cap.append(Final_Cap_Req2)
    
    
  else: 
    break 
      
## Attach Saved result to data frame      
Mortgage['EAD'] = EAD
Mortgage['Unpaid_Bal'] = Remaining_Bal
Mortgage['Provision'] = Mortgage_Provison
Mortgage['Capital_Requirement_IRB'] = Mortgage_IRB_Cap
Mortgage['IRB_RWA'] = Mortgage_IRB_RWA
Mortgage['Weight'] = Mortgage_weight
Mortgage['SA_RWA'] = Mortgage_SA_RWA
Mortgage['Capital_Requirement_SA'] = Mortgage_SA_Cap


## 1.2.4 Calculate Provison and Capital Requirements in the Retail Loan Business Line

In [ ]:
## Provision in the Retail Loan Line


## Function to Calculate the Remainning Balance for Loan
## We assumes the loan terms are in months 
def Rem_Bal_Loan (Principal, APR, terms, npaid):
    monthly_interest = APR/ 12
    payment_number = terms
    
    # Calculate fixed payment amount 
    monthly_payment = Principal*(monthly_interest/(1-(1+monthly_interest) 
                                                   **(- payment_number)))
    # give payments made
    rem_bal = rem_bal = Principal * (1 - ((1 + monthly_interest) ** npaid - 1) / 
                           ((1 + monthly_interest) ** payment_number - 1))
    
    return rem_bal


Stop3 = max(Loan.ID)
EAD = []
Loan_Provison = []
Loan_IRB_RWA = []
Loan_IRB_Cap = []
Loan_SA_RWA = []
Loan_SA_Cap = []

for index, row in Loan.iterrows():
  if row['ID'] <= Stop3:
    #Calculate EAD = outstanding balance 
    EAD_V = Rem_Bal_Loan (row['OriginalAmount'],row['YearlyInterestRate'],
                         row['Term'],row['CurrentInstallment'])
    EAD.append(EAD_V)
    
    #Apply floors in LGD and PD
    adjusted_PD = max(PDfloor, row['PD'])
    adjusted_LGD = max(LGDfloor, row['LGD'])
    
    #Calculate the Provision
    provinsion = EAD_V * adjusted_LGD * adjusted_PD
    Loan_Provison.append(provinsion)
    
    #Calculate F-IRB RWA
    Cap_Req =  CapReq_other_Retail(adjusted_LGD,adjusted_PD)
    Cap_Req = Cap_Req * EAD_V    
    RWA_V = 12.5 * Cap_Req
    Loan_IRB_RWA.append(RWA_V)
    
    
    #Calculate the IRB Capital Requirement
    Final_Cap_Req = RWA_V * cap_req_percent
    Loan_IRB_Cap.append(Final_Cap_Req)
    
    
    #Calculate Loan_weighed_RWA 
    #Weight_Retail_Loan is a predefined constant 75%
    W_RWA_V = EAD_V * Weight_Retail_Loan
    Loan_SA_RWA.append(W_RWA_V)
    
    
    #Calculate the SA Capital Requirement
    Final_Cap_Req2 = W_RWA_V * cap_req_percent
    Loan_SA_Cap.append(Final_Cap_Req2)

    
  else:
    break 
    
    
## Attach Saved result to data frame    
Loan['EAD'] = EAD
Loan['Provision'] = Loan_Provison
Loan['IRB_RWA'] = Loan_IRB_RWA
Loan['SA_RWA'] = Loan_SA_RWA
Loan['Capital_Requirement_SA'] = Loan_SA_Cap
Loan['Capital_Requirement_IRB'] = Loan_IRB_Cap
    
    
    
    
    
    
    
    
    
    

##1.2.5 Extract the Calculation Result

In [ ]:
#Extract Provision and Capital Requirement under F-IRB Approach for Bond 
print (sum(Bond['Bond_Provison']))
print (sum(Bond['Capital_Requirement_IRB']))


#Extract Provision and Capital Requirement under F-IRB Approach for Mortgages
print(sum(Mortgage['Provision']))
print(sum(Mortgage['Capital_Requirement_IRB']))



#Extract Provision and Capital Requirement under F-IRB Approach for Loans
print(sum(Loan['Provision']))
print(sum(Loan['Capital_Requirement_IRB']))




9858989107.772062
20092672282.9325
512264202.8959211
755794845.1596057
25753739.259888496
18518438.115086857


#1.3 Calculate The RWA under IRB Approach and Standerized Approach  

Because all th result was already calculated through the loop in Section 1.2, we just need to extract them out

In [ ]:
##Calculate the Total RWA under IRB Approach 
Total_Bond_IRB_RWA = sum(Bond.IRB_RWA)
Total_M_IRB_RWA = sum(Mortgage.IRB_RWA)
Total_Loan_IRB_RWA = sum(Loan.IRB_RWA)
Total_IRB_RWA = Total_Loan_IRB_RWA + Total_M_IRB_RWA + Total_Bond_IRB_RWA
print(Total_IRB_RWA)
print((Total_IRB_RWA* 11.5/100))

##Calculate the Total RWA under SA Approach 
Total_Bond_SA_RWA = sum(Bond.SA_RWA)
Total_M_SA_RWA = sum(Mortgage.SA_RWA)
Total_Loan_SA_RWA = sum(Loan.SA_RWA)
Total_SA_RWA = Total_Bond_SA_RWA + Total_M_SA_RWA + Total_Loan_SA_RWA
print (Total_SA_RWA)
print((Total_SA_RWA * 11.5/100))




181452048401.8026
20866985566.2073
44443105660.24057
5110957150.927666
